In [1]:
%matplotlib inline


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy.cosmology
import scipy.stats
import time
from useful_funcs import PAUSflux_to_ABmag

In [2]:
data = pd.read_pickle("../../Data/Object_zs_zp_181203_130401.pkl.gz")

In [3]:
data.columns.values
# data['production_id'].unique()

array(['ref_id', 'zb', 'odds', 'pz_width', 'zb_mean', 'chi2', 'n_band',
       'ebv', 'qz', 'best_run', 'production_id', 'paudm_id', 'ra', 'dec',
       'zp_gal', 'zl68_gal', 'zu68_gal', 'zl99_gal', 'zu99_gal', 'zp_sec',
       'dchi', 'imag', 'ei', 'i_auto', 'nbfilt', 'mod_gal', 'type',
       'umag', 'bmag', 'vmag', 'gmag', 'rmag', 'zmag', 'icmag', 'jmag',
       'kmag', 'mv', 'ebv_gal', 'ebv_int', 'acs_a_image', 'acs_b_image',
       'acs_theta_image', 'acs_mag_auto', 'acs_magerr_auto', 'zspec',
       'conf', 'f814w', 'zfits', 'r50', 'sersic_n_gim2d', '_col50',
       '_col54', 'n_coadd', 'run', '_col57', 'band0', 'band1', 'band2',
       'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9',
       'band10', 'band11', 'band12', 'band13', 'band14', 'band15',
       'band16', 'band17', 'band18', 'band19', 'band20', 'band21',
       'band22', 'band23', 'band24', 'band25', 'band26', 'band27',
       'band28', 'band29', 'band30', 'band31', 'band32', 'band33',
       'band34', '

In [4]:
def magAB_to_flux(mag):
    flux = 10**((mag -26 )/-2.5)
    return flux

In [5]:


median_qz = np.median(data['qz'])
cut_mask = (data['n_bands']==40) #& (data['zspec'] <= 1.2) & (data['zspec'] != 0)#(data['qz'] < median_qz) # 
extra_cols = ['ref_id','zb','pz_width','odds','qz','production_id','paudm_id','ra','dec','zspec','conf']

flux_sample = data.loc[cut_mask,'band0_flux':'band39_flux_error']
#error_sample = data.loc[:,'band0_flux_error':'band9_flux_error']
zs = data.loc[cut_mask,extra_cols]
mags = data.loc[cut_mask,['i_auto', 'umag', 'bmag', 'vmag', 'gmag', 'rmag', 'jmag', 'kmag']]


# magerrors = np.zeros(mags.values.shape)
magerrors = (magAB_to_flux(mags-0.02) - magAB_to_flux(mags+0.02))/2
magerrors.columns = ['i_auto_err','umag_err','bmag_err','vmag_err','gmag_err','rmag_err','jmag_err','kmag_err']
magerrors
mags = magAB_to_flux(mags)
# magerrors[:] = mags 
# mags


# magerrors = pd.DataFrame({'i_auto_err':magerrors,'umag_err':magerrors,'bmag_err':magerrors,'vmag_err':magerrors,\
#                          'gmag_err':magerrors,'rmag_err':magerrors,'jmag_err':magerrors,'kmag_err':magerrors},index=mags.index)


In [6]:
datacode = input("Data code?")
# mags = PAUSflux_to_ABmag(flux_sample).fillna(0)
complete = zs.join(mags).join(magerrors).join(flux_sample)
# complete.index.values.shape

shuffled = complete.sample(frac=1).reset_index(drop=True)
n_rows = shuffled.shape[0]
# print(np.floor(n_rows/3).astype(int))

# Splitting into 1 dataset
shuffled.to_csv("../../Data/ANNz_data"+datacode+".csv",index=False,na_rep=np.nan)

# Splitting into 3 datasets
# shuffled.iloc[:np.floor(n_rows*2/3).astype(int)].to_csv("../../Data/ANNz_data"+datacode+"a.csv",index=False,na_rep=np.nan)
# shuffled.iloc[np.floor(n_rows*2/3).astype(int):np.floor(n_rows*5/6).astype(int)].to_csv("../../Data/ANNz_data"+datacode+"b.csv",index=False,na_rep=np.nan)
# shuffled.iloc[np.floor(n_rows*5/6).astype(int):].to_csv("../../Data/ANNz_data"+datacode+"c.csv",index=False,na_rep=np.nan)



Data code?11


In [7]:
# shuffled.loc[13,'band0_flux']
# flux_sample.loc[flux_sample['band0_flux']==4.45071]
# data.loc[data['ref_id']==100477,['ref_id','i_auto', 'umag', 'bmag', 'vmag', 'gmag', 'rmag', 'jmag', 'kmag']]
# flux_sample
# data.loc[:,'band25':'band38']

string = ""
for i in range(25,38+1):
    string = string + " abs(band{0}_flux/band{0}_flux_error) +".format(i)

string

' abs(band25_flux/band25_flux_error) + abs(band26_flux/band26_flux_error) + abs(band27_flux/band27_flux_error) + abs(band28_flux/band28_flux_error) + abs(band29_flux/band29_flux_error) + abs(band30_flux/band30_flux_error) + abs(band31_flux/band31_flux_error) + abs(band32_flux/band32_flux_error) + abs(band33_flux/band33_flux_error) + abs(band34_flux/band34_flux_error) + abs(band35_flux/band35_flux_error) + abs(band36_flux/band36_flux_error) + abs(band37_flux/band37_flux_error) + abs(band38_flux/band38_flux_error) +'

In [7]:

string = ""
for i in complete.columns.values:
    string += "F:" + i + ";"
    
string

'F:ref_id;F:zb;F:pz_width;F:odds;F:qz;F:production_id;F:paudm_id;F:ra;F:dec;F:zspec;F:conf;F:i_auto;F:umag;F:bmag;F:vmag;F:gmag;F:rmag;F:jmag;F:kmag;F:i_auto_err;F:umag_err;F:bmag_err;F:vmag_err;F:gmag_err;F:rmag_err;F:jmag_err;F:kmag_err;F:band0_flux;F:band1_flux;F:band2_flux;F:band3_flux;F:band4_flux;F:band5_flux;F:band6_flux;F:band7_flux;F:band8_flux;F:band9_flux;F:band10_flux;F:band11_flux;F:band12_flux;F:band13_flux;F:band14_flux;F:band15_flux;F:band16_flux;F:band17_flux;F:band18_flux;F:band19_flux;F:band20_flux;F:band21_flux;F:band22_flux;F:band23_flux;F:band24_flux;F:band25_flux;F:band26_flux;F:band27_flux;F:band28_flux;F:band29_flux;F:band30_flux;F:band31_flux;F:band32_flux;F:band33_flux;F:band34_flux;F:band35_flux;F:band36_flux;F:band37_flux;F:band38_flux;F:band39_flux;F:band0_flux_error;F:band1_flux_error;F:band2_flux_error;F:band3_flux_error;F:band4_flux_error;F:band5_flux_error;F:band6_flux_error;F:band7_flux_error;F:band8_flux_error;F:band9_flux_error;F:band10_flux_error;F

In [9]:
string = ""
for i in complete.columns.values[27:]:
    string += i + ";"
    
string

'band0_flux;band1_flux;band2_flux;band3_flux;band4_flux;band5_flux;band6_flux;band7_flux;band8_flux;band9_flux;band10_flux;band11_flux;band12_flux;band13_flux;band14_flux;band15_flux;band16_flux;band17_flux;band18_flux;band19_flux;band20_flux;band21_flux;band22_flux;band23_flux;band24_flux;band25_flux;band26_flux;band27_flux;band28_flux;band29_flux;band30_flux;band31_flux;band32_flux;band33_flux;band34_flux;band35_flux;band36_flux;band37_flux;band38_flux;band39_flux;band0_flux_error;band1_flux_error;band2_flux_error;band3_flux_error;band4_flux_error;band5_flux_error;band6_flux_error;band7_flux_error;band8_flux_error;band9_flux_error;band10_flux_error;band11_flux_error;band12_flux_error;band13_flux_error;band14_flux_error;band15_flux_error;band16_flux_error;band17_flux_error;band18_flux_error;band19_flux_error;band20_flux_error;band21_flux_error;band22_flux_error;band23_flux_error;band24_flux_error;band25_flux_error;band26_flux_error;band27_flux_error;band28_flux_error;band29_flux_error

In [9]:
string = ""
for i in complete.columns.values[:11]:
    string += i + ";"
    
string

'ref_id;zb;pz_width;odds;qz;production_id;paudm_id;ra;dec;zspec;conf;'